In [1]:
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
from sklearn.preprocessing import scale


def populate(size: int, lower_bound, upper_bound, mean: float, std: float):

    # size: number of individuals in the population

    # lb: array containing the lower bound of talent value
    # lb = np.full(size, lower_bound)

    # ub: array containing the upper bound of talent value
    # ub = np.full(size, upper_bound)

    # mu: array containing the mean of talent distribution
    # mu = np.full(size, mean)

    # stdev: array containing the standard deviation of talent distribution
    # stdev = np.full(size, std)

    talent = np.zeros(size)

    talent = sp.stats.truncnorm.rvs((lower_bound - mean) / std,
                                    (upper_bound - mean) / std,
                                    loc=mean,
                                    scale=std,
                                    size=size)

    # talent = np.random.default_rng().normal(mean, std, size=size)

    mean = np.mean(talent)
    std = np.std(talent)

    talent_sort = np.sort(talent, 0, 'quicksort')
    talent_index = np.argsort(talent, 0, 'quicksort')

    return talent_sort, talent_index

def cpt_map(array: np.ndarray):
    '''Mapping from the position of an individual's random walk to their capital'''

    new_arr = 10 * (2**array)

    return new_arr

def evolution(talent: np.ndarray, time, unlucky_event, lucky_event, history=False, capital=False):
    '''If history=False (default behavior), returns a 1d array representing the population's final position.

       If history=True, returns a 2d array where:
            - The i-th row represent the time evolution of the i-th individual's position
            - The j-th column represents the population's position at the j-th iteration
            - The element (i, j) represents the position of the i-th individual at the j-th iteration
    '''

    rng = default_rng()

    pos = np.zeros((len(talent), time))

    # Initializing all individuals to a starting position of 0:

    np.place(pos[:, 0], mask=np.zeros(len(talent)) == 0, vals=0)

    if history:

        # Returns a 2d array where:
            # The i-th row represent the time evolution of the i-th individual's position
            # The j-th column represents the population's position at the j-th iteration
            # The element (i, j) represents the position of the i-th individual at the j-th iteration

        for i in range(time - 1):
            a = rng.uniform(0.0, 1.0, size=len(talent))
            b = rng.uniform(0.0, 1.0, size=len(talent))

            arr_source = pos[:, i]

            # Creating logical masks for each scenario:

                # Scenario 1: individual went through an unlucky event

            unlucky_mask = a < unlucky_event

                # Scenario 2: individual went through a lucky event and capitalized

            lucky_mask = ((a >= unlucky_event) &
                          (a < unlucky_event + lucky_event) &
                          (b < talent)
                          )

                # Scenario 3: individual didn't go through any events OR went through a lucky event and failed to capitalize:

            neutral_mask = ((a >= unlucky_event + lucky_event) |
                            ((a >= unlucky_event) &
                            (a < unlucky_event + lucky_event) &
                            (b > talent))
                            )

            # Upadting position of those in scenario 1:

            unlucky_vals = np.extract(unlucky_mask, arr_source) - 1

            np.place(pos[:, i + 1], mask=unlucky_mask, vals=unlucky_vals)

            # Upadting position of those in scenario 2:

            lucky_vals = np.extract(lucky_mask, arr_source) + 1

            np.place(pos[:, i + 1], mask=lucky_mask, vals=lucky_vals)

            # Upadting position of those in scenario 3:

            neutral_vals = np.extract(neutral_mask, arr_source)

            np.place(pos[:, i + 1], mask=neutral_mask, vals=neutral_vals)

            # Checking if the masks cover the entirety of the source array:

            full_mask = np.array((unlucky_mask, lucky_mask, neutral_mask))
            check = np.logical_or.reduce(full_mask)

            if np.all(check):
                pass
            else:
                raise ValueError('Failure to update position')

        if capital:
            # Returns both the position and the capital arrays

            cpt = cpt_map(pos)

            return pos, cpt

        else:
            # Default behavior
            # Returns the position array
            return pos

    else:

        # Default behavior
        # Returns a 1d array representing the population's final position

        iter = 0
        arr_source = pos[:, 0]

        while iter < time:

            a = rng.uniform(0.0, 1.0, size=len(talent))
            b = rng.uniform(0.0, 1.0, size=len(talent))

            # Creating logical masks for each scenario:

                # Scenario 1: individual went through an unlucky event
            unlucky_mask = a < unlucky_event

                # Scenario 2: individual went through a lucky event and capitalized
            lucky_mask = ((a >= unlucky_event) &
                          (a < unlucky_event + lucky_event) &
                          (b < talent)
                          )

                # Scenario 3: individual didn't go through any events OR went through a lucky event and failed to capitalize
            neutral_mask = ((a >= unlucky_event + lucky_event) |
                            ((a >= unlucky_event) &
                            (a < unlucky_event + lucky_event) &
                            (b > talent))
                            )

            # Upadting position of those in scenario 1:
            unlucky_vals = np.extract(unlucky_mask, arr_source) - 1

            np.place(arr_source, mask=unlucky_mask, vals=unlucky_vals)

            # Upadting position of those in scenario 2:
            lucky_vals = np.extract(lucky_mask, arr_source) + 1

            np.place(arr_source, mask=lucky_mask, vals=lucky_vals)

            # Upadting position of those in scenario 3:
            neutral_vals = np.extract(neutral_mask, arr_source)

            np.place(arr_source, mask=neutral_mask, vals=neutral_vals)

            # Checking if the masks cover the entirety of the source array:
            full_mask = np.array((unlucky_mask, lucky_mask, neutral_mask))
            check = np.logical_or.reduce(full_mask)

            if np.all(check):
                pass
            else:
                raise ValueError('Failure to update position')

            iter += 1

    if capital:
        # Returns both position and capital arrays

        cpt = cpt_map(arr_source)

        return arr_source, cpt

    else:
        # Returns only position array

        return arr_source

In [2]:
%%timeit -r 1 -n 1

UsageError: %%timeit is a cell magic, but the cell body is empty. Did you mean the line magic %timeit (single %)?


In [3]:
%%timeit -r 1 -n 1
import functions as f
from matplotlib import pyplot as plt
import numpy as np

np.set_printoptions(precision=3)

# iter_n: number of iterations to go through

iter_n = 80

# pop_n: number of individuals in the popoulation

pop_n = 1000

# lb: lower bound of talent
# ub: upper bound of talent

lb, ub = 0, 1

# mu: average value of the talent distribution
# std: standard deviation of the talent distribution

mu, std = 0.6, 0.1

# Creating a population with the desired parameters for talent distribution:

    # talent: array containing sorted values of talent

    # t_i: array containing the indices to the unsorted talent array, i.e:

        # talent[t_i[0]] is the talent of the first individual of the population

        # talent[t_i[j-1]] is the talent of the j-th individual of the population

        # talent[t_i[-1]] is the talent of the last individual of the population

talent, t_i = f.populate(pop_n, lb, ub, mu, std)

# le: chance for an individual to go through a lucky event
le = 0.03

# ue: chance for an individual to go through an unlucky event
ue = 0.03

# runs: number of runs to aggregate over
runs = 10000

# Initialize arrays to hold the position and the talent for the most succesful individual of each run:

# mst: Most Successful Talent (talent of the most succesful individual)
mst = np.empty(runs)

# msp: Most Successful Position (final position of the most succesful individual)
msp = np.empty(runs)

# Create an array to store values of talent and position for those who were overall successful:
successful = np.zeros((1, 2))

for i in range(runs):

    final_pos = f.evolution(talent, iter_n, ue, le)

    successful_per_run = np.column_stack((talent[final_pos > 0], final_pos[final_pos > 0]))

    successful = np.concatenate((successful, successful_per_run))

    mst[i] = talent[np.argmax(final_pos)]

    msp[i] = np.max(final_pos)

# msc: Most Successful Capital (final capital of the most succesful individual)
msc = f.cpt_map(msp)

1min 39s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [4]:
import functions as f
from matplotlib import pyplot as plt
import numpy as np

np.set_printoptions(precision=3)

# iter_n: number of iterations to go through

iter_n = 80

# pop_n: number of individuals in the popoulation

pop_n = 1000

# lb: lower bound of talent
# ub: upper bound of talent

lb, ub = 0, 1

# mu: average value of the talent distribution
# std: standard deviation of the talent distribution

mu, std = 0.6, 0.1

# Creating a population with the desired parameters for talent distribution:

    # talent: array containing sorted values of talent

    # t_i: array containing the indices to the unsorted talent array, i.e:

        # talent[t_i[0]] is the talent of the first individual of the population

        # talent[t_i[j-1]] is the talent of the j-th individual of the population

        # talent[t_i[-1]] is the talent of the last individual of the population

talent, t_i = f.populate(pop_n, lb, ub, mu, std)

# le: chance for an individual to go through a lucky event
le = 0.03

# ue: chance for an individual to go through an unlucky event
ue = 0.03

# runs: number of runs to aggregate over
runs = 10000

# Initialize arrays to hold the position and the talent for the most succesful individual of each run:

# mst: Most Successful Talent (talent of the most succesful individual)
mst = np.empty(runs)

# msp: Most Successful Position (final position of the most succesful individual)
msp = np.empty(runs)

# Create an array to store values of talent and position for those who were overall successful:
successful = np.zeros((1, 2))

In [5]:
%%timeit -r 100 -n 100
import functions as f
f.evolution(talent, iter_n, ue, le)

6.23 ms ± 97.9 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [6]:
%load_ext line_profiler

In [7]:
%lprun -f f.evolution f.evolution(talent, iter_n, ue, le)

Timer unit: 1e-06 s

Total time: 0.012985 s
File: /home/fran/tvl/functions.py
Function: evolution at line 49

Line #      Hits         Time  Per Hit   % Time  Line Contents
    49                                           def evolution(talent: np.ndarray, time, unlucky_event, lucky_event, history=False, capital=False):
    50                                               '''If history=False (default behavior), returns a 1d array representing the population's final position.
    51                                           
    52                                                  If history=True, returns a 2d array where:
    53                                                       - The i-th row represent the time evolution of the i-th individual's position
    54                                                       - The j-th column represents the population's position at the j-th iteration
    55                                                       - The element (i, j) represents th